1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [61]:

import pandas as pd

file_name = "./data/data_000637.txt"
df = pd.read_csv(file_name,  nrows = 1000000)

print (df)





        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0          1     0          123  3869200167        2374        26
1          1     0          124  3869200167        2374        27
2          1     0           63  3869200167        2553        28
3          1     0           64  3869200167        2558        19
4          1     0           64  3869200167        2760        25
...      ...   ...          ...         ...         ...       ...
999995     1     0           62  3869209011        2560        29
999996     1     0           51  3869209011        2561        12
999997     1     0           61  3869209011        2562         2
999998     1     0           50  3869209011        2563         4
999999     1     0           60  3869209011        2688        25

[1000000 rows x 6 columns]


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [58]:

df['BX_COUNTER'].max()


3563

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [57]:

df['ABSOLUTE_TIME'] = (df['ORBIT_CNT']-3869200167)*(3564*25)+(df['BX_COUNTER']-2374)*25+(df['TDC_MEAS']-26)*25/30
print(df)



        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0          1     0          123  3869200167        2374        26   
1          1     0          124  3869200167        2374        27   
2          1     0           63  3869200167        2553        28   
3          1     0           64  3869200167        2558        19   
4          1     0           64  3869200167        2760        25   
...      ...   ...          ...         ...         ...       ...   
999995     1     0           62  3869209011        2560        29   
999996     1     0           51  3869209011        2561        12   
999997     1     0           61  3869209011        2562         2   
999998     1     0           50  3869209011        2563         4   
999999     1     0           60  3869209011        2688        25   

        ABSOLUTE_TIME  
0        0.000000e+00  
1        8.333333e-01  
2        4.476667e+03  
3        4.594167e+03  
4        9.649167e+03  
...               ...  
999

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [59]:


df['ABSOLUTE_TIME'] = pd.to_datetime(df['ABSOLUTE_TIME']).dt.time
print(df)

     


        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0          1     0          123  3869200167        2374        26   
1          1     0          124  3869200167        2374        27   
2          1     0           63  3869200167        2553        28   
3          1     0           64  3869200167        2558        19   
4          1     0           64  3869200167        2760        25   
...      ...   ...          ...         ...         ...       ...   
999995     1     0           62  3869209011        2560        29   
999996     1     0           51  3869209011        2561        12   
999997     1     0           61  3869209011        2562         2   
999998     1     0           50  3869209011        2563         4   
999999     1     0           60  3869209011        2688        25   

          ABSOLUTE_TIME  
0              00:00:00  
1              00:00:00  
2       00:00:00.000004  
3       00:00:00.000004  
4       00:00:00.000009  
...            

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [55]:
print(df.groupby(['FPGA','TDC_CHANNEL'])['TDC_CHANNEL'].count().sort_values(ascending = False).head(3))

FPGA  TDC_CHANNEL
0     139            57878
      64             49109
      63             48604
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [53]:


nan = set(df['ORBIT_CNT'])
print(len(nan))




8841


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [54]:



tdc_139 = data[data['TDC_CHANNEL'] == 139]
tdc_139_orbits = set(tdc_139['ORBIT_CNT'])
print(len(tdc_139_orbits))




2392


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [60]:

FPGA0 = (df[df['FPGA']== 0]).groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
FPGA1 = (df[df['FPGA']==1]).groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
print(FPGA0,'\n')
print(FPGA1,'\n')


TDC_CHANNEL
1        933
2       1207
3       1156
4       1794
5       1096
       ...  
129        2
130       24
137       23
138       25
139    57878
Name: TDC_CHANNEL, Length: 124, dtype: int64 

TDC_CHANNEL
1      21622
2      24871
3      16725
4      20087
5      11375
       ...  
129       26
130       29
137       27
138       27
139    24799
Name: TDC_CHANNEL, Length: 132, dtype: int64 



9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.